In [ ]:

!pip3 install selenium webdriver-manager gspread oauth2client pandas


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options


chrome_options = Options()

chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)


chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

chrome_options.add_argument("--disable-blink-features=AutomationControlled")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

print("Stealth Browser Initialized. Now open the website manually once to see if it works!")

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)


sheet = client.open("Pixie_Event_Tracker").sheet1
print("Successfully connected to Google Sheets!")


In [ ]:
try:
   
    sheet_name = "Pixie_Event_Tracker" 
    spreadsheet = client.open(sheet_name)
    sheet = spreadsheet.sheet1
    print(f"Successfully connected to: {sheet_name}")
    
    
    print("Current Headers:", sheet.row_values(1))
    
except Exception as e:
    print(f"Error: {e}")
    print("\nHELPFUL TIP: Did you share the Google Sheet with the email found in your credentials.json?")


In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_and_sync(target_city):
    print(f"--- Starting Sync for {target_city.upper()} ---")
    
   
    url = f"https://in.bookmyshow.com/explore/events-{target_city.lower()}"
    driver.get(url)
    
   
    try:
        print("Waiting for event cards to appear...")
        
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="/events/"]'))
        )
    except:
        print("Timed out waiting for cards. Trying to scroll anyway...")

    
    driver.execute_script("window.scrollTo(0, 800);")
    time.sleep(2)
    
    
    all_values = sheet.get_all_values()
    existing_urls = [row[5] for row in all_values if len(row) > 5]
    
    
    all_links = driver.find_elements(By.TAG_NAME, 'a')
    event_data = []
    
    for link in all_links:
        href = link.get_attribute('href')
        if href and '/events/' in href and '/explore/' not in href:
            if href not in existing_urls and href not in [d[5] for d in event_data]:
                # Try to get the name from the card text OR the URL
                name = link.text.split('\n')[0]
                if not name:
                    name = href.split('/')[-2].replace('-', ' ').title()
                
                event_data.append([name, "Upcoming", "TBD", target_city.title(), "Events", href, "Live"])

    
    if event_data:
        sheet.append_rows(event_data)
        print(f"✅ SUCCESS: Added {len(event_data)} events to Google Sheets!")
    else:
        print("❌ Still found 0 events. Check if the Chrome window shows a pop-up or a 'No Events' message.")

scrape_and_sync("jaipur")